### Vector Store and retrievers
This will familiarize you with LangChain's Vector store and retriever abstractions. These abstractions are designed to support retrieval of data -- from (vector) databases and other sources -- for integration with LLM workflows. They are important for applications that fetch data to be reasoned over as part of model inference, as in the case of retrieval-agumented generation.

We will cover:
* Documents
* Vector stores
* Retrievers

### Documents
Langchain implements a Document abstraction, which is intented to represent a unit of text and associated metadata. It has two attributes:

page_content: a string representing the content;
metadata: a dict containing arbitarary metadata.
The metadata attribute can capture information about the source of the document, it's relationship to other documents, and other information. Note that an individual Document objective often represents a chunk of a larger document.

Let's generate some sample documents:

In [1]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source":"mammal-pets-doc"}
    ),
    Document(
        page_content="Cats are independent animals that often enjoy solitude, but can be affectionate.",
        metadata={"source":"mammal-pets-doc"}
    ),
    Document(
        page_content="Parrots are colorful birds that can mimic human speech and are highly social.",
        metadata={"source":"bird-pets-doc"}
    ),
    Document(
        page_content="Goldfish are popular aquarium pets that require a clean environment and proper care.",
        metadata={"source":"fish-pets-doc"}
    ),
]

In [2]:
documents

[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent animals that often enjoy solitude, but can be affectionate.'),
 Document(metadata={'source': 'bird-pets-doc'}, page_content='Parrots are colorful birds that can mimic human speech and are highly social.'),
 Document(metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular aquarium pets that require a clean environment and proper care.')]

In [3]:
import os
from langchain_groq import ChatGroq
from dotenv import load_dotenv
load_dotenv()
os.environ["HF_Token"] = os.getenv("HF_Token")

llm = ChatGroq(model="llama-3.1-8b-instant", temperature=0)
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001FB1C75E900>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001FB1C8963C0>, model_name='llama-3.1-8b-instant', temperature=1e-08, model_kwargs={}, groq_api_key=SecretStr('**********'))

In [4]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [5]:
## VectorStores
from langchain_chroma import Chroma

vectorstore = Chroma.from_documents(documents, embeddings)
vectorstore

In [7]:
vectorstore.similarity_search("Which pet is the most independent?")

[Document(id='5f37a1ad-0f92-40b4-aa4e-ab80f9aef476', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent animals that often enjoy solitude, but can be affectionate.'),
 Document(id='37532ac8-b5b9-43b3-9970-092ea2613345', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(id='d18db31c-80a6-4a40-bcda-55aee2c20405', metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular aquarium pets that require a clean environment and proper care.'),
 Document(id='04c5f2bd-697a-482a-8f64-24bfa061c72c', metadata={'source': 'bird-pets-doc'}, page_content='Parrots are colorful birds that can mimic human speech and are highly social.')]

In [8]:
## Async query
await vectorstore.asimilarity_search("Which pet is the most independent?")

[Document(id='5f37a1ad-0f92-40b4-aa4e-ab80f9aef476', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent animals that often enjoy solitude, but can be affectionate.'),
 Document(id='37532ac8-b5b9-43b3-9970-092ea2613345', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(id='d18db31c-80a6-4a40-bcda-55aee2c20405', metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular aquarium pets that require a clean environment and proper care.'),
 Document(id='04c5f2bd-697a-482a-8f64-24bfa061c72c', metadata={'source': 'bird-pets-doc'}, page_content='Parrots are colorful birds that can mimic human speech and are highly social.')]

In [9]:
vectorstore.similarity_search_with_score("cat")

[(Document(id='5f37a1ad-0f92-40b4-aa4e-ab80f9aef476', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent animals that often enjoy solitude, but can be affectionate.'),
  1.0698992013931274),
 (Document(id='37532ac8-b5b9-43b3-9970-092ea2613345', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
  1.574089765548706),
 (Document(id='04c5f2bd-697a-482a-8f64-24bfa061c72c', metadata={'source': 'bird-pets-doc'}, page_content='Parrots are colorful birds that can mimic human speech and are highly social.'),
  1.6840347051620483),
 (Document(id='d18db31c-80a6-4a40-bcda-55aee2c20405', metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular aquarium pets that require a clean environment and proper care.'),
  1.837925910949707)]

#### Retrievers
Langchain Vectorstore objects do not subclass Runnable, and so cannot immediately be integrated into Langchian Expression Language chains.

Langchian Retrievers are Runnables, so they implements a standard set of methods (e.g., synchornous and asynchornous invoke and batch operations) and are designed to be incorporated in LCEL chains.

We can create a simple version of this ourselves, without subclassing Retriever. If we choose what method we wish to use to retrieve documents, we can create a runnable easily. Below we will build one around the similarity_search method:

In [10]:
from typing import List

from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

retriever = RunnableLambda(vectorstore.similarity_search).bind(k=1)
retriever.batch(["cat","dog"])

[[Document(id='5f37a1ad-0f92-40b4-aa4e-ab80f9aef476', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent animals that often enjoy solitude, but can be affectionate.')],
 [Document(id='37532ac8-b5b9-43b3-9970-092ea2613345', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]]

Vectorstores implements an as_retriever method that will generate a Retriever, specially a VectorStoreRetriever. These retrievers includes specfic search_type and search_kwargs attributes that identify what methods of the underlying vector store to call, and how to parameterize them. For instance, we can replicate the above with the following:

In [12]:
retriever = vectorstore.as_retriever(
    search_type="similarity", 
    search_kwargs={"k":1}
)

retriever.batch(["cat","dog"])

[[Document(id='5f37a1ad-0f92-40b4-aa4e-ab80f9aef476', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent animals that often enjoy solitude, but can be affectionate.')],
 [Document(id='37532ac8-b5b9-43b3-9970-092ea2613345', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]]

In [ ]:
## Basic RAG chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message = """
Answer this question using the provided context only.
{question}

Context:
{context}
"""
prompt = ChatPromptTemplate.from_messages([("human", message)])

rag_chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | llm

response = rag_chain.invoke("tell me about cats?")
print(response.content)

Cats are independent animals that often enjoy solitude, but can be affectionate.
